<a href="https://colab.research.google.com/github/aaoiii/2024-ESAA-OB/blob/main/6%EC%A3%BC%EC%B0%A8_%ED%8C%8C%EB%A8%B8%EC%99%84_%EB%AC%B8%EC%84%9C%EA%B5%B0%EC%A7%91%ED%99%94%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 07 문서 군집화 소개와 실습 (Opinion Review 데이터 세트)

### **문서 군집화 개념**
: 비슷한 텍스트 구성의 문서를 군집화하는 것
- 동일한 군집에 속하는 문서를 같은 카테고리 소속으로 분류할 수 있음 (텍스트 분류 기반의 문서 부류와 유사)
- 학습 데이터가 필요없는 비지도학습 기반 (텍스트 분류 기반의 문서 분류와의 차이점)

### Opinion Review 데이터 세트를 이용한 문서 군집화 수행하기


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import glob, os

path = r'/content/drive/MyDrive/esaa/data/opinosis+opinion+frasl+review/topics'
all_files=glob.glob(os.path.join(path,"*data"))
filename_list=[]
opinion_text=[]

In [3]:
# 개별 파일의 파일명은 filename_list
# 개별 파일의 파일 내용은 DataFrame으로 로딩 후 다시 string으로 변환해 opinion_text list로 취합

for file_ in all_files:
  df = pd.read_table(file_, index_col=None, header=0, encoding='latin1')
  filename_ = file_.split('/')[-1]
  filename = filename_.split('.')[0]

  filename_list.append(filename)
  opinion_text.append(df.to_string())

document_df = pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head()

,filename,opinion_text
0,updates_garmin_nuvi_255W_gps,...
1,staff_swissotel_chicago,...
2,transmission_toyota_camry_2007,...
3,battery-life_ipod_nano_8gb,...
4,size_asus_netbook_1005ha,...


문서를 TF-IDF 형태로 피처 벡터화

In [5]:
from nltk.stem import WordNetLemmatizer

import nltk
import string
nltk.download('punkt')
nltk.download('wordnet')
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english',
                             ngram_range=(1,2), min_df=0.05, max_df=0.85)
# opinion_text 칼럼 값으로 피처 벡터화 수행
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


문서별 텍스트가 TF-IDF 변환된 피처 벡터화 행렬 데이터에 대해서 군집화를 수행해 어떤 문서끼리 군집되는지 확인
- 군집화 기법은 K-평균 사용

In [11]:
from sklearn.cluster import KMeans

# 5개 집합으로 군집화 수행
km_cluster=KMeans(n_clusters=5, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label=km_cluster.labels_
cluster_centers=km_cluster.cluster_centers_

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [12]:
document_df['cluster_label']=cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,updates_garmin_nuvi_255W_gps,...,0
1,staff_swissotel_chicago,...,2
2,transmission_toyota_camry_2007,...,4
3,battery-life_ipod_nano_8gb,...,1
4,size_asus_netbook_1005ha,...,1


cluster_label로 어떤 파일명으로 매칭되었는지 보면서 군집화 결과 확인해보기

In [13]:
document_df[document_df['cluster_label']==0].sort_values(by='filename')

,filename,opinion_text,cluster_label
15,accuracy_garmin_nuvi_255W_gps,...,0
7,directions_garmin_nuvi_255W_gps,...,0
8,satellite_garmin_nuvi_255W_gps,...,0
0,updates_garmin_nuvi_255W_gps,...,0
10,voice_garmin_nuvi_255W_gps,...,0


In [15]:
document_df[document_df['cluster_label']==1].sort_values(by='filename')

,filename,opinion_text,cluster_label
26,battery-life_amazon_kindle,...,1
3,battery-life_ipod_nano_8gb,...,1
13,battery-life_netbook_1005ha,...,1
21,buttons_amazon_kindle,...,1
5,display_garmin_nuvi_255W_gps,...,1
29,eyesight-issues_amazon_kindle,...,1
16,features_windows7,...,1
31,fonts_amazon_kindle,...,1
14,keyboard_netbook_1005ha,...,1
32,navigation_amazon_kindle,...,1


In [16]:
document_df[document_df['cluster_label']==2].sort_values(by='filename')

,filename,opinion_text,cluster_label
50,bathroom_bestwestern_hotel_sfo,...,2
46,free_bestwestern_hotel_sfo,...,2
41,location_bestwestern_hotel_sfo,...,2
24,location_holiday_inn_london,...,2
49,parking_bestwestern_hotel_sfo,...,2
23,price_holiday_inn_london,...,2
25,room_holiday_inn_london,...,2
38,rooms_bestwestern_hotel_sfo,...,2
27,rooms_swissotel_chicago,...,2
39,staff_bestwestern_hotel_sfo,...,2


In [17]:
document_df[document_df['cluster_label']==3].sort_values(by='filename')

,filename,opinion_text,cluster_label
36,food_holiday_inn_london,...,3
33,food_swissotel_chicago,...,3
44,service_bestwestern_hotel_sfo,...,3
20,service_holiday_inn_london,...,3
30,service_swissotel_hotel_chicago,...,3


In [18]:
document_df[document_df['cluster_label']==4].sort_values(by='filename')

,filename,opinion_text,cluster_label
40,comfort_honda_accord_2008,...,4
35,comfort_toyota_camry_2007,...,4
22,gas_mileage_toyota_camry_2007,...,4
42,interior_honda_accord_2008,...,4
28,interior_toyota_camry_2007,...,4
43,mileage_honda_accord_2008,...,4
48,performance_honda_accord_2008,...,4
47,quality_toyota_camry_2007,...,4
45,seats_honda_accord_2008,...,4
2,transmission_toyota_camry_2007,...,4


군집 중심 개수를 5개에서 3개로 낮춰서 3개 그룹으로 군집화해보기

In [19]:
from sklearn.cluster import KMeans

# 3개 집합으로 군집화 수행
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

# 소속 군집을 cluster_label 칼럼으로 할당하고 cluster_label 값으로 정렬
document_df['cluster_label'] = cluster_label
document_df.sort_values(by='cluster_label')

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,filename,opinion_text,cluster_label
25,room_holiday_inn_london,...,0
46,free_bestwestern_hotel_sfo,...,0
44,service_bestwestern_hotel_sfo,...,0
41,location_bestwestern_hotel_sfo,...,0
39,staff_bestwestern_hotel_sfo,...,0
38,rooms_bestwestern_hotel_sfo,...,0
36,food_holiday_inn_london,...,0
33,food_swissotel_chicago,...,0
30,service_swissotel_hotel_chicago,...,0
27,rooms_swissotel_chicago,...,0


- cluster 0: 포터블 전자기기
- cluster 1: 호텔
- cluster 2: 자동차

## 군집별 핵심 단어 추출하기
- 각 군집을 구성하는 핵심 단어가 어떤 것이 있는지 확인
- KMeans 객체는 각 군집을 구성하는 단어 피처가 군집의 중심을 기준으로 얼마나 가깝게 위치해있는지 clusters_centers_ 속성 제공
  - 배열 값으로 제공 (개별 군집, 개별 피처)

In [20]:
cluster_centers=km_cluster.cluster_centers_
print('cluster_centers shape:', cluster_centers.shape)
print(cluster_centers)

cluster_centers shape: (3, 4611)
[[0.         0.00099499 0.00174637 ... 0.         0.00183397 0.00144581]
 [0.         0.00092551 0.         ... 0.         0.         0.        ]
 [0.01005322 0.         0.         ... 0.00706287 0.         0.        ]]


군집이 3개, word피처가 4611개로 구성
- 각 행의 배열 값은 각 군집 내의 피처의 위치가 개별 중심과 얼마나 가까운가를 상대 값으로 나타냄
- 0~1 사이, 1에 가까울 수록 중심과 가까운 값을 의미

**cluster_centers_**
- 넘파이의 ndarray
- ndarray argsort()[:,::-1]를 이용하면 cluster_centers 배열 내 값이 큰 순으로 정렬된 위치 인덱스 값 반환 (큰 값을 가진 배열 내 위치 인덱스 값을 반환)
- 위치 인덱스 값 : 핵심 단어 피처의 이름을 출력하기 위해 필요


In [21]:
# 군집별 top n 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명을 반환함
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num,
                        top_n_features=10):
  cluster_details={}

  # cluster_centers array 값이 큰 순으로 정렬된 인덱스 값을 반환
  # 군집 중심점 별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함
  centroid_feature_ordered_ind=cluster_model.cluster_centers_.argsort()[:, ::-1]

  # 개별 군집별로 반복하며 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명 입력
  for cluster_num in range(clusters_num):
    # 개별 군집별 정보를 담을 데이터 초기화
    cluster_details[cluster_num]={}
    cluster_details[cluster_num]['cluster']=cluster_num

    # cluster_centers_.argsort()로 구한 인덱스를 이용해 top n 피처 단어를 구함
    top_feature_indexes=centroid_feature_ordered_ind[cluster_num, :top_n_features]
    top_features=[feature_names[ind] for ind in top_feature_indexes]

    # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함
    top_feature_values=cluster_model.cluster_centers_[cluster_num,
                                                      top_feature_indexes].tolist()

    # cluster_details 딕셔너리 객체에 개별 군집별 핵심단어와 중심위치 상댓값, 해당 파일명 입력
    cluster_details[cluster_num]['top_features']=top_features
    cluster_details[cluster_num]['top_features_value']=top_feature_values
    filenames=cluster_data[cluster_data['cluster_label']==cluster_num]['filename']
    filenames=filenames.values.tolist()

    cluster_details[cluster_num]['filenames']=filenames

  return cluster_details

- dictionary를 원소로 가지는 리스트인 cluster_details 반환

In [22]:
def print_cluster_details(cluster_details):
  for cluster_num, cluster_detail in cluster_details.items():
    print('###### Cluster {0}'.format(cluster_num))
    print('Top features:',cluster_detail['top_features'])
    print('Review 파일명 :',cluster_detail['filenames'][:7])
    print('=============================================')

In [24]:
feature_names=tfidf_vect.get_feature_names_out()
cluster_details=get_cluster_details(cluster_model=km_cluster,cluster_data=document_df,
                                    feature_names=feature_names,clusters_num=3,top_n_features=10)
print_cluster_details(cluster_details)

###### Cluster 0
Top features: ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'clean', 'price', 'parking']
Review 파일명 : ['staff_swissotel_chicago', 'service_holiday_inn_london', 'price_holiday_inn_london', 'location_holiday_inn_london', 'room_holiday_inn_london', 'rooms_swissotel_chicago', 'service_swissotel_hotel_chicago']
###### Cluster 1
Top features: ['interior', 'seat', 'mileage', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'performance', 'quality']
Review 파일명 : ['transmission_toyota_camry_2007', 'gas_mileage_toyota_camry_2007', 'interior_toyota_camry_2007', 'comfort_toyota_camry_2007', 'comfort_honda_accord_2008', 'interior_honda_accord_2008', 'mileage_honda_accord_2008']
###### Cluster 2
Top features: ['screen', 'battery', 'keyboard', 'battery life', 'life', 'kindle', 'direction', 'video', 'size', 'voice']
Review 파일명 : ['updates_garmin_nuvi_255W_gps', 'battery-life_ipod_nano_8gb', 'size_asus_netbook_1005ha', 'display_garmin_nuvi_255W_gps', 